In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv(r'C:\Users\Jouke\Documents\evedata-logger\output\market_data_with_names_merged.csv', parse_dates=['date'])

In [ ]:
# Sum volume for each item per day (if not already per day)
daily_volume = df.groupby(['type_id', 'date'])['volume'].sum().reset_index()

# Calculate average daily volume for each item
avg_daily_volume = daily_volume.groupby('type_id')['volume'].mean().reset_index()
avg_daily_volume.columns = ['type_id', 'avg_daily_volume']

In [ ]:
# Spread is highest - lowest price each day
df['spread'] = df['highest'] - df['lowest']

# Average spread per item
avg_spread = df.groupby('type_id')['spread'].mean().reset_index()
avg_spread.columns = ['type_id', 'avg_bid_ask_spread']